# Data Augmentation
In this notebook, 3 Variants of SMOTE are compared for data augmentation. The training dataset is being scaled to 2000 observations.
Manual noise is induced for synthetic samples in order to improve robustness and prevent overfitting.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, KMeansSMOTE
import joblib
from sklearn.model_selection import train_test_split

## Functions and Utils

In [ ]:
# Create a custom colormap
colors = [
    (0.945, 0.980, 0.733),
    (0.263, 0.671, 0.702),
    (0.137, 0.294, 0.620),
]

custom_cmap = LinearSegmentedColormap.from_list("custom_diverging", colors, N=256)

# Custom formatter for correlation values
def custom_fmt(val):
    abs_val = abs(val)
    if abs_val == 1.0:
        return "1.0"
    else:
        return f".{int(abs_val * 100):02d}"

# Function to create and save correlation heatmap
def plot_correlation_heatmap(df, title, filename):
    if 'performance_class' in df.columns:
        data = df.drop(columns=["performance_class"])
    else:
        data = df
    
    corr_matrix = data.corr()
    plt.figure(figsize=(12, 12))
    heatmap = sns.heatmap(
        corr_matrix,
        mask=np.triu(np.ones_like(corr_matrix, dtype=bool), k=1),
        annot=True,
        fmt="",
        annot_kws={"size": 8},
        cmap=custom_cmap,
        vmin=-1, vmax=1,
        center=0,
        linewidths=0.2,
        cbar_kws={"shrink": 0.8}
    )
    
    for text in heatmap.texts:
        text_value = float(text.get_text().replace('−', '-'))
        text.set_text(custom_fmt(text_value))
    
    plt.title(title, fontsize=14)
    plt.subplots_adjust(bottom=0.3)
    
    plt.savefig(filename, bbox_inches='tight', dpi=300)
    plt.close()
    
    return corr_matrix

# Function to calculate correlation difference metrics
def calculate_correlation_difference(original_corr, augmented_corr):
    # Calculate absolute differences
    diff_matrix = np.abs(original_corr - augmented_corr)
    
    # Calculate mean absolute difference (excluding diagonal)
    mask = ~np.eye(original_corr.shape[0], dtype=bool)
    mean_abs_diff = diff_matrix.values[mask].mean()
    
    # Calculate max absolute difference
    max_abs_diff = diff_matrix.values[mask].max()
    
    return {
        'mean_abs_diff': mean_abs_diff,
        'max_abs_diff': max_abs_diff,
        'diff_matrix': diff_matrix
    }

# Function to plot feature distributions
def plot_feature_distributions(original_df, augmented_df, method_name):
    if 'performance_class' in original_df.columns:
        original_features = original_df.drop(columns=['performance_class'])
        augmented_features = augmented_df.drop(columns=['performance_class'])
    else:
        original_features = original_df
        augmented_features = augmented_df
    
    # Select a subset of features if there are too many
    features_to_plot = original_features.columns[:min(10, len(original_features.columns))]
    
    fig, axes = plt.subplots(len(features_to_plot), 2, figsize=(15, 4*len(features_to_plot)))
    
    for i, feature in enumerate(features_to_plot):
        # Original data distribution
        sns.histplot(original_features[feature], kde=True, color='royalblue', ax=axes[i, 0])
        axes[i, 0].set_title(f"{feature} (Original)")
        
        # Augmented data distribution
        sns.histplot(augmented_features[feature], kde=True, color='orange', ax=axes[i, 1])
        axes[i, 1].set_title(f"{feature} ({method_name})")
    
    plt.tight_layout()
    plt.savefig(f"../data/figures/feature_distributions_{method_name}.pdf", dpi=300)
    plt.close()
    

def apply_smote_with_noise(X, y, augmenter, noise_level=0.05, random_state=42):
    """
    Apply SMOTE and add random noise to the synthetic samples
    
    Parameters:
    -----------
    X : DataFrame
        Features
    y : Series
        Target labels
    augmenter : object
        SMOTE, BorderlineSMOTE, or KMeansSMOTE object
    noise_level : float
        Standard deviation of Gaussian noise as a fraction of feature range
    random_state : int
        Random seed for reproducibility
    
    Returns:
    --------
    X_resampled : DataFrame
        Features with original and noisy synthetic samples
    y_resampled : Series
        Corresponding labels
    """
    # Record original samples to identify synthetic ones later
    original_indices = X.index.tolist()
    
    # Apply SMOTE
    X_smote, y_smote = augmenter.fit_resample(X, y)
    
    # Convert to DataFrame and Series to maintain column names and other metadata
    if not isinstance(X_smote, pd.DataFrame):
        X_smote = pd.DataFrame(X_smote, columns=X.columns)
    if not isinstance(y_smote, pd.Series):
        y_smote = pd.Series(y_smote, name=y.name)
    
    synthetic_mask = ~X_smote.index.isin(original_indices)
    synthetic_indices = X_smote.index[synthetic_mask]
    
    # Generate noise proportional to each feature's range
    feature_ranges = X.max() - X.min()
    np.random.seed(random_state)
    noise = np.random.normal(
        0, 
        noise_level, 
        size=(len(synthetic_indices), X.shape[1])
    ) * feature_ranges.values
    
    # Add noise to synthetic samples
    X_smote.loc[synthetic_indices] += noise
    
    return X_smote, y_smote

## Run Data Augmentation

In [ ]:
# Main function to run the augmentation
def run_augmentation_experiment(input_file):
    # ---- LOAD DATASET ----

    df = pd.read_csv(input_file)
    X = df.drop(columns=["performance_class"])
    y = df["performance_class"]
    
    # ---- SPLIT DATASET ----
    
    print("Splitting dataset into train/test sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    
    # Save split datasets for consistent splits
    print("Saving train/test splits...")
    joblib.dump(X_train, "../data/splits/X_train.pkl")
    joblib.dump(y_train, "../data/splits/y_train.pkl")
    joblib.dump(X_test, "../data/splits/X_test.pkl")
    joblib.dump(y_test, "../data/splits/y_test.pkl")
    
    print("Train/test split saved successfully!")
    
    # Create a DataFrame from the training data for correlation analysis
    df_train = pd.DataFrame(X_train, columns=X.columns)
    df_train["performance_class"] = y_train
    
    # Save original training data correlation matrix
    original_corr = plot_correlation_heatmap(
        X_train, 
        "Original Training Data Correlation Matrix", 
        "../data/figures/original_train_correlation_heatmap.pdf"
    )
    
    # Augmentation methods
    augmentation_methods = {
        'SMOTE_k5': SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=42),
        'BorderlineSMOTE': BorderlineSMOTE(sampling_strategy='auto', random_state=42),
        'KMeansSMOTE': KMeansSMOTE(sampling_strategy='auto', cluster_balance_threshold=0.1, random_state=42)
    }
    
    results = {}
    augmented_dfs = {}
    
    # Run each augmentation method on the training data
    for method_name, augmenter in augmentation_methods.items():
        print(f"\nRunning {method_name} on training data only...")
        
        try:
            # ---- Step 1: Initial augmentation to balance classes ----
            
            X_train_resampled_1, y_train_resampled_1 = apply_smote_with_noise(
                X_train, y_train, augmenter, noise_level=0.05, random_state=42
            )
            df_train_balanced = pd.DataFrame(X_train_resampled_1, columns=X_train.columns)
            df_train_balanced["performance_class"] = y_train_resampled_1
                        
            print(f"Training dataset size after initial {method_name}:", df_train_balanced.shape)
            print(y_train_resampled_1.value_counts())
                        
            # ---- Step 2: Large scale augmentation to achieve 2000 samples in total ----
            
            target_count_per_class = 1000
            sampling_strategy_large = {}
            for cls in y_train_resampled_1.unique():
                sampling_strategy_large[cls] = target_count_per_class
                        
            augmenter_params = {}
            if method_name == 'KMeansSMOTE':
                augmenter_params['cluster_balance_threshold'] = 0.1
                        
            if method_name == 'SMOTE_k5':
                augmenter_large = SMOTE(sampling_strategy=sampling_strategy_large, k_neighbors=5, random_state=44, **augmenter_params)
            elif method_name == 'BorderlineSMOTE':
                augmenter_large = BorderlineSMOTE(sampling_strategy=sampling_strategy_large, random_state=44, **augmenter_params)
            elif method_name == 'KMeansSMOTE':
                augmenter_large = KMeansSMOTE(sampling_strategy=sampling_strategy_large, random_state=44, **augmenter_params)
                        
            X_train_balanced = df_train_balanced.drop(columns=["performance_class"])
            y_train_balanced = df_train_balanced["performance_class"]
            X_train_resampled_large, y_train_resampled_large = apply_smote_with_noise(
                X_train_balanced, y_train_balanced, augmenter_large, noise_level=0.05, random_state=44
            )
                        
            df_train_large = pd.DataFrame(X_train_resampled_large, columns=X_train.columns)
            df_train_large["performance_class"] = y_train_resampled_large
                        
            print(f"Large training dataset size after {method_name}:", df_train_large.shape)
            print(y_train_resampled_large.value_counts())
            
            # ---- SAVE AUGMENTED TRAINING DATA ----
            
            joblib.dump(X_train_resampled_large, f"../data/splits/X_train_augmented_{method_name}.pkl")
            joblib.dump(y_train_resampled_large, f"../data/splits/y_train_augmented_{method_name}.pkl")
            print(f"Augmented training data saved as pkl files for {method_name}")
                        
            # ---- SAVE AUGMENTED CORRELATION MATRIX ----
            
            augmented_corr = plot_correlation_heatmap(
                df_train_large.drop(columns=["performance_class"]),
                f"{method_name} Augmented Training Data Correlation Matrix",
                f"../data/figures/{method_name}_train_correlation_heatmap.pdf"
            )
                        
            # ---- CALCULATE CORRELATION DIFFERENCES COMPARED TO ORIGINAL FEATURE CORRELATIONS ----
            
            diff_results = calculate_correlation_difference(original_corr, augmented_corr)
                        
            # ---- SAVE RESULTS ----
            
            results[method_name] = diff_results
            augmented_dfs[method_name] = df_train_large
                        
            # ---- PLOT CORRELATION DIFFERENCE MATRICES ----
            
            plt.figure(figsize=(12, 10))
            heatmap = sns.heatmap(
                diff_results['diff_matrix'],
                mask=np.triu(np.ones_like(diff_results['diff_matrix'], dtype=bool), k=1),
                annot=True,
                fmt="",
                annot_kws={"size": 8},
                cmap=custom_cmap,
                vmin=0,
                vmax=0.5
            )
            for text in heatmap.texts:
                text_value = float(text.get_text().replace('−', '-'))
                text.set_text(custom_fmt(text_value))
            plt.title(f'Correlation Difference: {method_name}\nMean: {diff_results["mean_abs_diff"]:.4f}, Max: {diff_results["max_abs_diff"]:.4f}')
            plt.tight_layout()
            plt.savefig(f"../data/figures/{method_name}_diff_heatmap.pdf", dpi=300)
            plt.close()
            
        except Exception as e:
            print(f"Error with {method_name}: {str(e)}")
    
    # ---- SAVE CORRELATION DIFFERENCE METRICS ----
    
    metrics_df = pd.DataFrame({
        'Method': list(results.keys()),
        'Mean_Absolute_Difference': [results[method]['mean_abs_diff'] for method in results],
        'Max_Absolute_Difference': [results[method]['max_abs_diff'] for method in results]
    })
    
    metrics_df.to_csv("../data/correlation_difference_metrics.csv", index=False)
    print("\nCorrelation difference metrics saved to 'correlation_difference_metrics.csv'")
    
    # Find the best method
    if results:
        best_method = min(results.items(), key=lambda x: x[1]['mean_abs_diff'])[0]
        print(f"\nBest method based on correlation preservation: {best_method}")
        
        # ---- PLOT FEATURE DISTRIBUTION FOR THE BEST SMOTE AUGMENTATION ALGORITHM ----
        if best_method in augmented_dfs:
            df_train_for_plot = pd.DataFrame(X_train, columns=X_train.columns)
            df_train_for_plot["performance_class"] = y_train
            
            plot_feature_distributions(df_train_for_plot, augmented_dfs[best_method], best_method)
            print(f"Feature distribution plot saved for {best_method}")
            
            best_df = augmented_dfs[best_method]
            X_train_augmented_best = best_df.drop(columns=["performance_class"])
            y_train_augmented_best = best_df["performance_class"]
            
            # ---- SAVE BEST AUGMENTED TRAINING DATA ----
            
            joblib.dump(X_train_augmented_best, f"../data/splits/X_train_augmented_best.pkl")
            joblib.dump(y_train_augmented_best, f"../data/splits/y_train_augmented_best.pkl")
            print(f"Best augmented training data ({best_method}) saved as pickle files")
    
    return results, augmented_dfs

# ---- RUN THE CODE ----
if __name__ == "__main__":
    results, augmented_dfs = run_augmentation_experiment("../data/features/filtered_labeled_feature_matrix.csv")